import os
from pathlib import Path

p = Path.cwd()
os.chdir(p.parent)
os.getcwd()

In [1]:
import os
from pathlib import Path

p = Path.cwd()
os.chdir(p.parent)
os.getcwd()

'/root/repos/lightning/HousePricing'

In [3]:
from dotenv import dotenv_values
import os

envs = ["secret.env", "test.env"]

for fenv in envs:
    file = os.path.join("env", fenv)
    config = dotenv_values(file)  # load sensitive variables
    print(config.keys())
    for c, v in config.items():
        os.environ[c] = v

odict_keys(['WANDB_API_KEY'])
odict_keys(['WANDB_NAME', 'WANDB_NOTEBOOK_NAME'])


In [4]:
import wandb
import os

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
# Preparing data to be used
from src.data import HousePricingDataModule

data_module = HousePricingDataModule()
data_module.prepare_data()

[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...


In [9]:
import pandas as pd

artifacts = pd.read_csv("artifacts.csv")
artifacts

,RunExperiment,SaveModelDirectory
0,Jennifer_4WW5OXKQ,./lightning_logs/22sevs1r
1,Joseph_62CMZWRP,./lightning_logs/rr56f2ic
2,Faith_S24EZHLE,./lightning_logs/ab55wxop


In [13]:
experiments = artifacts["RunExperiment"]
directories = artifacts["SaveModelDirectory"]

artifacts = dict(zip(experiments, directories))
artifacts

{'Jennifer_4WW5OXKQ': './lightning_logs/22sevs1r',
 'Joseph_62CMZWRP': './lightning_logs/rr56f2ic',
 'Faith_S24EZHLE': './lightning_logs/ab55wxop'}

In [16]:
import os
from os.path import join, basename
from lightning import Trainer
import torch

from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from src.model import NeuralNetwork

project_name = os.environ["WANDB_NAME"]

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
in_features = data_module.data_features()

config = {
    "accelerator": accelerator,
    "in_features": in_features,
}

test_results = dict()

for artifact_experiment, artifact_dir in artifacts.items():

    wandb.init(
        job_type="testing",
        name=artifact_experiment,
        project=project_name,
        config=config,
    )

    model = NeuralNetwork.load_from_checkpoint(
        checkpoint_path=join(artifact_dir, "model.ckpt"),
        input_size=wandb.config["in_features"],
    )

    logger = WandbLogger(checkpoint_name=artifact_experiment)

    trainer = Trainer(
        accelerator=wandb.config["accelerator"], logger=logger, log_every_n_steps=1
    )

    test_result = trainer.test(model, datamodule=data_module)
    test_result[artifact_experiment] = test_result

    wandb.finish()

FileNotFoundError: [Errno 2] No such file or directory: '/root/repos/lightning/HousePricing/lightning_logs/22sevs1r/model.ckpt'